<a href="https://colab.research.google.com/github/Kapil-Bhatt/GAN_GUJ_CHARACTER/blob/master/Guj_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import os
from PIL import Image
from matplotlib import pyplot

In [0]:
#image size is 112*112 pixels
img_rows = 112
img_cols = 112
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

In [0]:
#generator function input latent dimention 100
def build_generator():
    model = Sequential()
    model.add(Dense(512 * 28 * 28, activation="relu", input_dim=latent_dim))
    model.add(Reshape((28, 28, 512)))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(channels, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))
    model.summary()
    noise = Input(shape=(latent_dim,))
    img = model(noise)
    return model

In [0]:
#descriminator function 
def build_discriminator():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    img = Input(shape=img_shape)
    validity = model(img)
    return model

In [60]:
#compile discriminator 
optimizer = Adam(0.0002, 0.5)
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 56, 56, 32)        320       
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 56, 56, 32)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 28, 28, 64)        18496     
_________________________________________________________________
zero_padding2d_5 (ZeroPaddin (None, 29, 29, 64)        0         
_________________________________________________________________
batch_normalization_21 (Batc (None, 29, 29, 64)        256       
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 29, 29, 64)       

In [61]:
generator = build_generator()
z = Input(shape=(100,))
img = generator(z)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 401408)            40542208  
_________________________________________________________________
reshape_5 (Reshape)          (None, 28, 28, 512)       0         
_________________________________________________________________
up_sampling2d_9 (UpSampling2 (None, 56, 56, 512)       0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 56, 56, 128)       589952    
_________________________________________________________________
batch_normalization_24 (Batc (None, 56, 56, 128)       512       
_________________________________________________________________
activation_13 (Activation)   (None, 56, 56, 128)       0         
_________________________________________________________________
up_sampling2d_10 (UpSampling (None, 112, 112, 128)   

In [0]:
#combined of discriminator and generator GAN model 
discriminator.trainable = False
valid = discriminator(img)
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

In [0]:
# train the model
def train(epochs, batch_size, save_interval):
    #make directory
    os.makedirs('images', exist_ok=True)

    #read our csv file from drive
    read=pd.read_csv("/content/drive/My Drive/Dataset/img_pixels_update.csv").values
    X_train=np.array
    X_train= read.reshape(216,112,112)

    # Rescale -1 to 1
    X_train = X_train / 127.5 - 1.
    X_train = np.expand_dims(X_train, axis=3)

    # Adversarial gives label 1 to valid images and 0 to fake 
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Select a random real images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_imgs = X_train[idx]

        # Sample noise and generate a batch of fake images
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_imgs = generator.predict(noise)

        # Train the discriminator
        D_loss_real = discriminator.train_on_batch(real_imgs, valid)
        D_loss_fake = discriminator.train_on_batch(fake_imgs, fake)
        D_loss = 0.5 * np.add(D_loss_real, D_loss_fake)

        # Train the generator
        g_loss = combined.train_on_batch(noise, valid)

        # If at save interval
        if epoch % save_interval == 0:
            # Print the progress
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, D_loss[0], 100 * D_loss[1], g_loss))
            # Save generated image samples
            save_imgs(epoch)

In [0]:
def save_imgs(epoch):
    r, c = 4, 4
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig("images/characters_%d.png" % epoch)
    plt.close()

In [67]:
train(epochs=2000, batch_size=2, save_interval=50)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.160586, acc.: 100.00%] [G loss: 2.949872]
50 [D loss: 0.630973, acc.: 75.00%] [G loss: 3.342306]
100 [D loss: 1.049036, acc.: 0.00%] [G loss: 0.774753]
150 [D loss: 0.732542, acc.: 50.00%] [G loss: 3.322635]
200 [D loss: 0.497535, acc.: 75.00%] [G loss: 1.903042]
250 [D loss: 3.717418, acc.: 0.00%] [G loss: 0.515563]
300 [D loss: 1.040916, acc.: 50.00%] [G loss: 2.176764]
350 [D loss: 0.044292, acc.: 100.00%] [G loss: 1.559094]
400 [D loss: 0.726980, acc.: 50.00%] [G loss: 2.682159]
450 [D loss: 0.265348, acc.: 75.00%] [G loss: 3.675426]
500 [D loss: 1.927059, acc.: 25.00%] [G loss: 4.617197]
550 [D loss: 1.590790, acc.: 50.00%] [G loss: 0.910067]
600 [D loss: 0.664665, acc.: 75.00%] [G loss: 1.855151]
650 [D loss: 0.982043, acc.: 50.00%] [G loss: 0.353055]
700 [D loss: 0.405560, acc.: 100.00%] [G loss: 4.605043]
750 [D loss: 0.655857, acc.: 50.00%] [G loss: 2.563517]
800 [D loss: 1.401794, acc.: 50.00%] [G loss: 0.692639]
850 [D loss: 0.595949, acc.: 75.00%] [G loss: 1.27

In [0]:
os.makedirs('model_weights', exist_ok=True)
generator.save_weights('model_weights/generator_weights.h5')
discriminator.save_weights('model_weights/discriminator_weights.h5')
combined.save_weights('model_weights/combined_weights.h5')